## RISK ANALYSIS - SPARQL

### What is the flood risk in Newark? 
As a flood risk manager, I want to understand the flood risk of Newark and the associated uncertainties. This flood risk assessment should combine information from all available data sources. Any inconsistencies in the modelling assumption should be made visible as well potential intervention strategies to reduce and mitigate flood risk.
### Questions of interest
__Damage Estimation__
* What is the expected annual damage for Newark?
* How are the damages separated by certain confounding factors?
>* Type of property
>* Gauges for which the properties are linked 
>* Flood warning areas
>* Time of the modelling studies used in the damage estimations
* How do the estimates vary through considering a number of different estimation techniques and uncertainties?
>* Interpolation schemes 
>* Different assumptions
>>* Onset of flooding
>> * Truncation error
>> * Uncertainty 	
* Flood Defence Schemes (Cost Benefit Analysis)
>* How much savings would flood defences with particular design criteria (protection to a particular return period) save? 
>* Since the hydraulic model was run, have any new flood defences been installed? 
>* Are there any proposed flood defence schemes in Newark? Does this analysis suggest that the flood defence is financially justified? 
* Observed Events
>* Can retrieve information about these events from Section 19 reports or from other sources? 
>* Can we retrieve estimates of the return period and estimates of the damage of these observed event
>>* Can this unstructured information provide additional information in the calculation of the expected annual damage
>>* Is this new observed event greater than the design events considered in the hydraulic model? If so highlight the models that need to be rerun 
* Provenance of Models 
>* If a model is out of date, there should be the option to re run the model (it is unlikely that we will have the time to implement but we could produce a guided example through using the open source software HEC-RAS)
>* If two potential models exist for the same location (this currently does not exist in the Communities at Risk data set), the query should reveal the differences between the two models and reveal the provenance of a particular model 
* How do I want the results to be presented?
>* Where possible all results should be presented spatially? 
>* All possible available metadata should be returned along with the assumptions made in the damage calculations 
>* Possible interventions strategies to reduce flood risk should be suggested with supporting evidence 
>* Any models that might be out of date should be highlighted 

### Setup SPARQL parameters for triple store and data formatting

In [1]:
%endpoint http://localhost:8890/sparql/
%format JSON
%display table

Endpoint set to: http://localhost:8890/sparql/
Return format: JSON
Display: table

In [2]:
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix dbp: <http://dbpedia.org/property/>
prefix dbo: <http://dbpedia.org/ontology/>
prefix dbr: <http://dbpedia.org/resource/>

select distinct  ?postTown ?lat ?long ?comment ?population
where  { 
    service <http://dbpedia.org/sparql/> {
        dbr:Newark-on-Trent dbp:postTown ?postTown ;
                            dbp:latitude ?lat ;
                            dbp:longitude ?long ;
                            rdfs:comment ?comment ;
                            dbo:populationTotal ?population .
        filter langMatches(lang(?comment),"en")
    }
}

postTown,lat,long,comment,population
NEWARK,53.0765,-0.81,"Newark-on-Trent or Newark /ˈnjuːək/ is a market town in Nottinghamshire in the East Midlands of England. It stands on the River Trent, the A1 (on the route of the ancient Great North Road), and the East Coast Main Line railway. The origins of the town are possibly Roman as it lies on an important Roman road, the Fosse Way. The town grew around Newark Castle, now ruined, and a large marketplace, now lined with historic buildings, and was a centre for the wool and cloth trade. In the English Civil War, it was besieged by Parliamentary forces and had to be relieved by Prince Rupert in a battle known as the Relief of Newark.",27700


In [ ]:
prefix pd:  <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select  (sum(xsd:decimal(?ead_mean)) as ?ead_mean)
from <http://ensembleprojects.org/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/damage#0>
where {
    ?property pd:posttown "NEWARK" .
    ?property pd:jbapropRef ?ref .
    ?newProperty pd:propRef ?ref . 
    ?newProperty pd:ead_mean ?ead_mean .
}

In [6]:
select ?s ?o ?p
where {
    ?s ?o ?p .
}

s,o,p
http://www.openlinksw.com/virtrdf-data-formats#default-iid,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
http://www.openlinksw.com/virtrdf-data-formats#default-iid-nullable,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
http://www.openlinksw.com/virtrdf-data-formats#default-iid-nonblank,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
http://www.openlinksw.com/virtrdf-data-formats#default-iid-nonblank-nullable,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
http://www.openlinksw.com/virtrdf-data-formats#default,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
http://www.openlinksw.com/virtrdf-data-formats#default-nullable,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
http://www.openlinksw.com/virtrdf-data-formats#sql-varchar,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
http://www.openlinksw.com/virtrdf-data-formats#sql-varchar-nullable,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
http://www.openlinksw.com/virtrdf-data-formats#sql-varchar-dt,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
http://www.openlinksw.com/virtrdf-data-formats#sql-varchar-dt-nullable,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat


### EAD for Newark

In [3]:
prefix pd:  <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select  (sum(xsd:decimal(?ead_mean)) as ?ead_mean)
from <http://ensembleprojects.org/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/damage#0>
where {
    ?property pd:posttown "NEWARK" .
    ?property pd:jbapropRef ?ref .
    ?newProperty pd:propRef ?ref . 
    ?newProperty pd:ead_mean ?ead_mean .
}


ead_mean
""


### EAD for Newark by Property Type

In [4]:
prefix pd:  <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select  ?jbapropertyType (sum(xsd:decimal(?ead_mean)) as ?ead_mean)
from <http://ensembleprojects.org/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/damage#0>
where {
    ?property pd:posttown "NEWARK" ;
              pd:jbapropertyType ?jbapropertyType ;
              pd:jbapropRef ?ref .
    ?newProperty pd:propRef ?ref ; 
                 pd:ead_mean ?ead_mean .
}
group by ?jbapropertyType

jbapropertyType,ead_mean


### EAD for Newark by Gauge Station

In [9]:
prefix pd:  <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select  ?station_name (sum(xsd:decimal(?ead_mean)) as ?ead_mean)
from <http://ensembleprojects.org/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/damage#0>
where {
    ?property pd:posttown "NEWARK" ;
              pd:station_name ?station_name ;
              pd:jbapropRef ?ref .
    ?newProperty pd:propRef ?ref ; 
                 pd:ead_mean ?ead_mean .
}
group by ?station_name

station_name,ead_mean
NORTH MUSKHAM,412898.175655102
COLSTON BASSETT,2564.85756758
WHITEWATER BRIDGE,0
TORKSEY,1096263.335348682
COLWICK,1256108.634237965
"MANSFIELD, THE DYKES",223090.66940382
CHURCH WARSOP,16.45146409


### EAD for Newark by Flood Warning Areas

In [ ]:
prefix pd:  <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select  ?fwa_name (sum(xsd:decimal(?ead_mean)) as ?ead_mean)
from <http://ensembleprojects.org/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/damage#0>
where {
    ?property pd:posttown "NEWARK" ;
              pd:fwa_name ?fwa_name ;
              pd:jbapropRef ?ref .
    ?newProperty pd:propRef ?ref ; 
                 pd:ead_mean ?ead_mean .
}
group by ?fwa_name

### EAD for Newark by Flood Warning Zones

In [ ]:
prefix pd:  <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select 
    (sum(xsd:decimal(?fz1)) as ?fz1) 
    (sum(xsd:decimal(?fz2)) as ?fz2)
    (sum(xsd:decimal(?fz3)) as ?fz3)
from <http://ensembleprojects.org/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/damage#0>
where {
    {
        ?property pd:posttown "NEWARK" ;
              pd:fz2 'NO';
              pd:fz3 'YES';
              pd:jbapropRef ?ref .
        ?newProperty pd:propRef ?ref ; 
                 pd:ead_mean ?fz3 .        
    }
    union
    {
        ?property pd:posttown "NEWARK" ;
              pd:fz2 'YES';
              pd:fz3 'YES';
              pd:jbapropRef ?ref .
        ?newProperty pd:propRef ?ref ; 
                 pd:ead_mean ?fz2 .         
    }
    union
    {
        ?property pd:posttown "NEWARK" ;
              pd:fz2 'NO';
              pd:fz3 'NO';
              pd:jbapropRef ?ref .
        ?newProperty pd:propRef ?ref ; 
                 pd:ead_mean ?fz1 .         
    }
}


### EAD for Newark by Model Study

In [3]:
prefix pd:  <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select  ?model (sum(xsd:decimal(?ead_mean)) as ?ead_mean)
from <http://ensembleprojects.org/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/damage#0>
where {
    ?property pd:posttown "NEWARK" ;
              pd:model ?model ;
              pd:jbapropRef ?ref .
    ?newProperty pd:propRef ?ref ; 
                 pd:ead_mean ?ead_mean .
}
group by ?model

model,ead_mean
Tidal Trent SFRM Model Fluvial Dominated_ Mott Macdonald_ 2015,1509161.511003784
River Trent and Tributaries SFRM2_ Halcrow_ July 2011,1256175.653147835
River Maun Flood Risk Mapping_ JBA_ March 2007,223090.66940382
River Meden Flood Risk Mapping Study_ JBA_ June 2008,16.45146409
Flood Modelling of the River Smite_ JBA_ March 2012,2497.83865771


### Estimation uncertainty - Onset of Flooding

In [ ]:
prefix pd:  <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select
    (sum(xsd:decimal(?ead_mean_opt)) as ?ead_mean_opt)
    (sum(xsd:decimal(?ead_mean_mid)) as ?ead_mean_mid)
    (sum(xsd:decimal(?ead_mean_pess)) as ?ead_mean_pess)
where {
    graph <http://ensembleprojects.org/floodrisk/property> {
        ?property pd:posttown "NEWARK" .
        ?property pd:jbapropRef ?jbapropRef
    }
    graph <http://ensembleprojects.org/graph/floodrisk/damage#0> {
        ?newProperty pd:propRef ?jbapropRef .
        ?newProperty pd:ead_mean ?ead_mean_opt .
    }
    graph <http://ensembleprojects.org/graph/floodrisk/damage#1> {
        ?newProperty pd:propRef ?jbapropRef .
        ?newProperty pd:ead_mean ?ead_mean_pess .
    }
    graph <http://ensembleprojects.org/graph/floodrisk/damage#2> {
        ?newProperty pd:propRef ?jbapropRef .
        ?newProperty pd:ead_mean ?ead_mean_mid .
    }
}

### Estimation uncertainty - Truncation

In [ ]:
prefix pd:  <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix dd:  <http://ensembleprojects.org/ns/floodrisk/damage_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select
    (sum(xsd:decimal(?ead_mean_opt)) as ?ead_mean_opt)
    (sum(xsd:decimal(?ead_mean_mid)) as ?ead_mean_mid)
    (sum(xsd:decimal(?ead_mean_pess)) as ?ead_mean_pess)
where {
    graph <http://ensembleprojects.org/floodrisk/property> {
        ?property pd:posttown "NEWARK" .
        ?property pd:jbapropRef ?jbapropRef
    }
    graph <http://ensembleprojects.org/graph/floodrisk/damage#5> {
        ?newProperty pd:propRef ?jbapropRef .
        ?newProperty pd:opt ?ead_mean_opt .
        ?newProperty pd:mid ?ead_mean_mid .
        ?newProperty pd:pess ?ead_mean_pess .
    }
}

### Estimation uncertainty - Monte Carlo Simulation

In [ ]:
prefix pd:  <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select
    (sum(xsd:decimal(?ead_mean)) as ?ead_mean)
    (sum(xsd:decimal(?ead_lowerTi)) as ?ead_lowerTi)
    (sum(xsd:decimal(?ead_upperTi)) as ?ead_upperTi)
where {
        graph <http://ensembleprojects.org/floodrisk/property> {
        ?property pd:posttown "NEWARK" .
        ?property pd:jbapropRef ?jbapropRef
    }
    graph <http://ensembleprojects.org/graph/floodrisk/damage#3> {
        ?newProperty pd:propRef ?jbapropRef .
        ?newProperty pd:ead_mean ?ead_mean .
        ?newProperty pd:ead_lowerTi ?ead_lowerTi .
        ?newProperty pd:ead_upperTi ?ead_upperTi 
    }
}

### Observed Events - EAD for Colwick following observed event

In [ ]:
prefix pd:  <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix dd:  <http://ensembleprojects.org/ns/floodrisk/damage_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select
    (sum(xsd:decimal(?ead)) as ?ead)
where {
    graph <http://ensembleprojects.org/floodrisk/property> {
        ?property pd:posttown "NEWARK" .
        ?property pd:station_name "COLWICK" .
        ?property pd:jbapropRef ?jbapropRef
    }
    graph <http://ensembleprojects.org/graph/floodrisk/damage#4> {
        ?newProperty pd:propRef ?jbapropRef .
        ?newProperty pd:ead ?ead 
    }
}

### Observed Events - Metadata

In [ ]:
prefix ed: <http://ensembleprojects.org/ns/floodrisk/event_data#>
select ?location ?rp ?properties ?source ?date
where {
    graph <http://ensembleprojects.org/graph/floodrisk/events> {
        ?s ed:location ?location ;
           ed:rp ?rp ;
           ed:properties ?properties ;
           ed:url ?source ;
           ed:date ?date .
    }
}

### Observed Events - EAD from C@R data

In [ ]:
prefix pd:  <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select  (sum(xsd:decimal(?ead_mean)) as ?ead_mean)
from <http://ensembleprojects.org/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/damage#0>
where {
    ?property pd:posttown "NEWARK" .
    ?property pd:station_name "COLWICK" .
    ?property pd:jbapropRef ?ref .
    ?newProperty pd:propRef ?ref . 
    ?newProperty pd:ead_mean ?ead_mean .
}

### Observed Events - Map data

In [ ]:
prefix gd:    <http://ensembleprojects.org/ns/floodrisk/gauge_data#>
prefix rt:    <http://environment.data.gov.uk/flood-monitoring/def/core/>
prefix geo:   <http://www.w3.org/2003/01/geo/wgs84_pos#>

select ?lat ?long ?label
from <http://ensembleprojects.org/floodrisk/gauge>
from <http://ensembleprojects.org/floodrisk/property>
from <http://environment.data.gov.uk/flood-monitoring/id/stations.rdf>
where {
    ?local_gauge gd:station_name "COLWICK" ;
       gd:site_reference ?site_reference .
    ?ea_gauge rt:stationReference ?site_reference ;
              geo:lat ?lat ;
              geo:long ?long ;
              rdfs:label ?label .
}

### Observed Events - Properties

In [ ]:
prefix pd:  <http://ensembleprojects.org/ns/floodrisk/property_data#>

select  ?jbapropRef ?bngeast ?bngnorth
from <http://ensembleprojects.org/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/damage#0>
where {
    ?property pd:posttown "NEWARK" ;
              pd:station_name "COLWICK" ;
              pd:jbapropRef ?jbapropRef ;
              pd:bngeast ?bngeast ;
              pd:bngnorth ?bngnorth .
}

### Observed Events - Property Details

In [ ]:
prefix pd:  <http://ensembleprojects.org/ns/floodrisk/property_data#>

select  ?q2 ?q5 ?q10 ?q20 ?q50 ?q75 ?q100 ?q200 ?q1000
from <http://ensembleprojects.org/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/damage#0>
where {
    ?s pd:jbapropRef '51183' ;
       pd:q2_existingDamageMean ?q2 ;
       pd:q5_existingDamagelMean ?q5 ;
       pd:q10_existingDamageMean ?q10 ;
       pd:q20_existingDamageMean ?q20 ;
       pd:q50_existingDamageMean ?q50 ;
       pd:q75_existingDamageMean ?q75 ;
       pd:q100_existingDamageMean ?q100 ;
       pd:q200_existingDamageMean ?q200 ;
       pd:q1000_existingDamagelMean ?q1000 
}

In [ ]:
prefix gd: <http://ensembleprojects.org/ns/floodrisk/gauge_data#>

select distinct ?station_name ?site_reference
from <http://ensembleprojects.org/floodrisk/gauge>
where {
    ?s gd:station_name "COLWICK" ;
       gd:site_reference ?site_reference .
}

In [ ]:
prefix pd:  <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select
    (sum(xsd:decimal(?q0)) as ?Q0)
    (sum(xsd:decimal(?q2)) as ?Q2)
    (sum(xsd:decimal(?q5)) as ?Q5)
    (sum(xsd:decimal(?q10)) as ?Q10)
    (sum(xsd:decimal(?q20)) as ?Q20)
    (sum(xsd:decimal(?q50)) as ?Q50)
    (sum(xsd:decimal(?q75)) as ?Q75)
    (sum(xsd:decimal(?q100)) as ?Q100)
    (sum(xsd:decimal(?q200)) as ?Q200)
    (sum(xsd:decimal(?q1000)) as ?Q1000)
from <http://ensembleprojects.org/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/property#0>
where {
    ?property pd:posttown "NEWARK" ;
              pd:jbapropRef ?ref .
    ?newProperty pd:propRef ?ref ; 
                 pd:q0 ?q0 ;
                 pd:q2 ?q2 ;
                 pd:q5 ?q5 ;
                 pd:q10 ?q10 ;
                 pd:q20 ?q20 ;
                 pd:q50 ?q50 ;
                 pd:q75 ?q75 ;
                 pd:q100 ?q100 ;
                 pd:q200 ?q200 ;
                 pd:q1000 ?q1000 .
}

In [ ]:
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix dbo: <http://dbpedia.org/ontology/>

select distinct ?subject ?comment ?population
where  { 
    service <http://dbpedia.org/sparql> {
        ?s rdfs:label "Newark-on-Trent"@en .
        ?s rdfs:comment ?comment .
        ?s dbo:population ?population
        filter langMatches(lang(?comment),"en")
    }
}

In [ ]:
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix dbp: <http://dbpedia.org/property/>
prefix dbo: <http://dbpedia.org/ontology/>
prefix dbr: <http://dbpedia.org/resource/>
prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>

select distinct  ?postTown ?lat ?long ?comment ?population
where  { 
    service <http://dbpedia.org/sparql> {
        dbr:Newark-on-Trent dbp:postTown ?postTown ;
                            dbp:latitude ?lat ;
                            dbp:longitude ?long ;
                            rdfs:comment ?comment ;
                            dbo:populationTotal ?population .
        filter langMatches(lang(?comment),"en")
    }
}

In [ ]:
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix dbp: <http://dbpedia.org/property/>
prefix dbo: <http://dbpedia.org/ontology/>
prefix dbr: <http://dbpedia.org/resource/>
prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>

select  distinct ?p
where  { 
    service <http://dbpedia.org/sparql> {
        dbr:Newark-on-Trent ?p ?o
        filter(regex(?p, 'ontology', 'i'))
    }
}

In [ ]:
select distinct ?p
from <http://ensembleprojects.org/floodrisk/property>
where {
    ?s ?p ?o .
    filter(regex (str(?p), 'fz', 'i'))
}

### Triple Store Overview

#### Named Graphs

In [ ]:
SELECT DISTINCT ?g 
WHERE {
  GRAPH ?g { ?s ?p ?o }
}

#### Graph Details

##### Raw data graphs generated from Communities @ Risk spreadsheet 
Gauge Worksheet:
http://ensembleprojects.org/floodrisk/gauge <br />
Property Worksheet:
http://ensembleprojects.org/floodrisk/property

##### New Graph created to store splitting of model string into distinct 'name', 'organisation' and 'date' triples
http://ensembleprojects.org/gauge 

##### OWL / RDFS model for generic gauge flow return period class and sub-classses
OWL / RDFS Model: http://ensembleprojects.org/owl/gaugemodel <br />
Data to populate ontology: http://ensembleprojects.org/floodrisk/gaugeleveldata 

##### OWL  / RDFS model for generic property damage return period class and sub-classses
http://ensembleprojects.org/owl/propertymodel

##### New Statistical Damage Estimates
Collection of named property damage estimate graphs:
http://ensembleprojects.org/floodrisk/damageestimates <br />
A named property damage estimate graph:
http://ensembleprojects.org/floodrisk/damage#0

##### Environment Agency Real-Time Flood APIs
http://environment.data.gov.uk/flood-monitoring/id/measures.rdf <br />
http://environment.data.gov.uk/flood-monitoring/data/readings.rdf


In [ ]:
SELECT ?s ?p ?o
FROM <http://ensembleprojects.org/owl/propertymodel>
WHERE {?s ?p ?o}

### Example 1
Find all flow rate return periods from gauging station "Ashford". In the current C@R data this is only possible via a regular expression, where here we have added semantic information manually to represent the fact that the different return periods (e.g. q2, q5, q10, etc) are a sub-class of the generic 'q' return period.

<img src="sw1.png" width="600" height="400" />

In [ ]:
prefix g:       <http://ensembleprojects.org/ns/floodrisk/gauge#>
prefix gd:      <http://ensembleprojects.org/ns/floodrisk/gauge_data#>
prefix p:       <http://ensembleprojects.org/ns/floodrisk/property#>
prefix pd:      <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix fr:      <http://ensembleprojects.org/floodrisk/>

select ?name ?flow ?val
from <http://ensembleprojects.org/floodrisk/gauge>
from <http://ensembleprojects.org/floodrisk/property>
where {
        g:0 gd:station_name ?name .
        g:0 ?flow ?val .
        ?flow a fr:q .
}


### Example 2
Example 1 showed how we could manually add in class information to a dataset. This example shows how we can also use RDFS/OWL to add semantic information to existing datasets. We first of create an RDFS/OWL class model of the property damage mean and then sub-class off that the various different return periods. We then elaborate the existing data to instantiate the different return periods.

<img src='sw3.png' width="600" height="400" align="left" /><br><br>

<img src='sw4.png' width="600" height="400" align="left" />


In [ ]:
DEFINE input:inference <http://ensembleprojects.org/owl/propertymodel>

PREFIX pm: <http://ensembleprojects.org/ns/owl/propertymodel/>
PREFIX p: <http://ensembleprojects.org/ns/floodrisk/property#>
PREFIX pd: <http://ensembleprojects.org/ns/floodrisk/property_data#>
SELECT ?damageMean ?height
FROM <http://ensembleprojects.org/floodrisk/property>
WHERE { 
  p:0 ?damageMean ?height .
  ?damageMean a pm:q_existingDamageMeanClass .
}

### Example 3
Shows how we add in new semantics using our own predicates. We have added in manually an ordering predicate for return values allowing queries for 'less than' or 'greater than' particular return periods. The example below queries for return periods less than q50.

<img src="sw2.png" width="600" height="400" />

In [ ]:
prefix g:       <http://ensembleprojects.org/ns/floodrisk/gauge#>
prefix gd:      <http://ensembleprojects.org/ns/floodrisk/gauge_data#>
prefix p:       <http://ensembleprojects.org/ns/floodrisk/property#>
prefix pd:      <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix fr:      <http://ensembleprojects.org/floodrisk/>

select ?name ?flow ?val 
from <http://ensembleprojects.org/floodrisk/gauge>
from <http://ensembleprojects.org/floodrisk/property>
where {
        g:0 gd:station_name ?name .
        g:0 ?flow ?val .
        ?flow a fr:q .
        ?flow fr:lessThan fr:q50 .
}

In [ ]:
prefix damage:     <http://ensembleprojects.org/ns/floodrisk/damage#>
prefix damageData: <http://ensembleprojects.org/ns/floodrisk/damage_data#>

select *
from <http://ensembleprojects.org/floodrisk/damageestimates> 
where {
 ?s ?p ?o
}

### Example 4
This example shows the integration of different data sources. In the current C@R datasets the geographical location of gauging stations is not available (it is for properties). To display gauging stations and related properties on a map we link the gauge station id available in the Communities at Risk dataset with the gauge station id available from the Environment Agency real-time flooding APIs. This API does provide lat and long information for the station and we use that to place it on a map.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import folium
import EN2LatLon
   
query = """
prefix g:     <http://ensembleprojects.org/ns/floodrisk/gauge#>
prefix gd:    <http://ensembleprojects.org/ns/floodrisk/gauge_data#>
prefix p:     <http://ensembleprojects.org/ns/floodrisk/property#>
prefix pd:    <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
prefix rt:    <http://environment.data.gov.uk/flood-monitoring/def/core/>
prefix geo:   <http://www.w3.org/2003/01/geo/wgs84_pos#>

select *
from <http://ensembleprojects.org/floodrisk/gauge>
from <http://ensembleprojects.org/floodrisk/property>
from <http://environment.data.gov.uk/flood-monitoring/id/stations.rdf>
where {
        ?property pd:station_name "ASHFORD" .
        ?property pd:bngeast ?bngeast .
        ?property pd:bngnorth ?bngnorth .
        ?property pd:q2_existingDamageMean ?q2_existingDamageMean .
        ?property pd:q5_existingDamagelMean ?q5_existingDamageMean .
        ?property pd:q10_existingDamageMean ?q10_existingDamageMean .
        ?property pd:q20_existingDamageMean ?q20_existingDamageMean .
        ?property pd:q50_existingDamageMean ?q50_existingDamageMean .
        ?property pd:property_thresh ?property_thresh .
        ?property pd:threshold_source ?threshold_source .
        
        ?station gd:station_name "ASHFORD" .
        ?station gd:site_reference ?site_reference .
        ?s rt:stationReference ?site_reference .
        ?s rt:catchmentName ?catchmentName .
        ?s rt:riverName ?riverName .
        ?s geo:lat ?lat .
        ?s geo:long ?long .
        ?s rdfs:label ?label .
}
limit 10
"""

# Assumes OpenLink Virtuoso sparql endpoint running at:
sparql_endpoint = "http://localhost:8890/sparql"

sparql = SPARQLWrapper(sparql_endpoint)
sparql.setReturnFormat(JSON)

sparql.setQuery(query)
results = sparql.query().convert()

stationPopup = "Name: {}<br />River Name: {}<br />Catchment Name: {}".format(
        results["results"]["bindings"][0]["label"]["value"],
        results["results"]["bindings"][0]["riverName"]["value"],
        results["results"]["bindings"][0]["catchmentName"]["value"]
)

lat = results["results"]["bindings"][0]["lat"]["value"]
lon = results["results"]["bindings"][0]["long"]["value"]
label = results["results"]["bindings"][0]["label"]["value"]
    
map_osm = folium.Map(location=[float(lat), float(lon)],zoom_start=11)
folium.Marker(location=[float(lat), float(lon)],
                  icon=folium.Icon(color='blue', prefix='fa', icon='tachometer'),
                  popup=stationPopup).add_to(map_osm) 

for result in results["results"]["bindings"]:
    lat, lon = EN2LatLon.OSGB36toWGS84(float(result["bngeast"]["value"]), float(result["bngnorth"]["value"]))
    propertyPopup = \
        """Threshold: {}<br />Threshold Source: {}<br />
           Q2 Existing Damage Mean: {}<br />
           Q5 Existing Damage Mean: {}<br />
           Q10 Existing Damage Mean: {}<br />
           Q20 Existing Damage Mean: {}<br />
           Q50 Existing Damage Mean: {}""".format(
                result["property_thresh"]["value"],
                result["threshold_source"]["value"],
                result["q2_existingDamageMean"]["value"],
                result["q5_existingDamageMean"]["value"],
                result["q10_existingDamageMean"]["value"],
                result["q20_existingDamageMean"]["value"],
                result["q50_existingDamageMean"]["value"])

    folium.Marker(location=[lat, lon],
        icon=folium.Icon(color='lightred', prefix='fa', icon='home'), 
        popup=propertyPopup).add_to(map_osm)
    
map_osm

### Example 5

Convert to a function and use inteface elements of Jupyter to allow selection from a dropdown box

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import gauges

station_list = gauges.stations()
interact(gauges.gaugeStation, name=station_list)

### Example 6
Find number of properties with threshold measurement via LiDAR or Survey. We might use this to assess uncertainty around LiDAR measurements for example and provide justification for a future threshold survey.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql_query = """
prefix g:       <http://ensembleprojects.org/ns/floodrisk/gauge#>
prefix gd:      <http://ensembleprojects.org/ns/floodrisk/gauge_data#>
prefix p:       <http://ensembleprojects.org/ns/floodrisk/property#>
prefix pd:      <http://ensembleprojects.org/ns/floodrisk/property_data#>

select ?lidar_count ?survey_count
from <http://ensembleprojects.org/floodrisk/gauge>
from <http://ensembleprojects.org/floodrisk/property>
where
{
    {select (count(distinct ?property_l) as ?lidar_count)
    where {
        {?property_l pd:threshold_source "LiDAR" .}
    }}
    union 
    {select (count(distinct ?property_s) as ?survey_count)
    where {
        {?property_s pd:threshold_source "Survey" .}
    }}
}
"""
# Assumes OpenLink Virtuoso sparql endpoint running at:
sparql_endpoint = "http://localhost:8890/sparql"

sparql = SPARQLWrapper(sparql_endpoint)
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for r in results['results']['bindings']:
    for k in list(r):
        print(k, r[k]['value'])


### Example 7
The Communities at Risk datset currently stores information about models used as a text string (e.g. 'River Meden Flood Risk Mapping Study_ JBA_ June 2008'). The SPARQL code below unpacks this string into its constituent parts (Study Name, Organisation, Date) then stores this information in the triple store.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import calendar

sparql_query = """
SELECT DISTINCT ?s ?o
FROM <http://ensembleprojects.org/floodrisk/gauge>
WHERE {
  ?s <http://ensembleprojects.org/ns/floodrisk/gauge_data#station_model> ?o .
}
"""
sparql = SPARQLWrapper(sparql_endpoint)
sparql.setReturnFormat(JSON)
sparql.setMethod('POST')
sparql.setQuery(sparql_query)
results = sparql.query().convert()

modelDict = {}
i = 0
for result in results['results']['bindings']:
    modelDict[result['o']['value']] = i
    i = i+1

# Assumes OpenLink Virtuoso sparql endpoint running at:
sparql_endpoint = "http://localhost:8890/sparql"

months = {v: k for k,v in enumerate(calendar.month_name)}
for result in results['results']['bindings']:
    modelName, modelCompany, modelDate = result['o']['value'].split('_')
    modelName = modelName.strip()
    modelCompany = modelCompany.strip()
    modelDate = modelDate.strip()
    dateElements = modelDate.split()
    if len(dateElements) > 1:
      dateMonth = str(months[dateElements[0]]).zfill(2)
      dateYear = dateElements[1]
      dateString = "{}-{}-01".format(dateYear, dateMonth)
    else:
      dateYear = dateElements[0]
      dateString = "{}-01-01".format(dateYear)
    modelID = modelDict[result['o']['value']]
    sparql_update = """ 
            PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
            INSERT DATA {{
              GRAPH <http://ensembleprojects.org/gauge> {{ 
                <{}> <http://ensembleprojects.org/ns/floodrisk/gauge_data#station_model_id> <http://ensembleprojects.org/ns/floodrisk/gauge_data#station_model_id#{}> .
                <http://ensembleprojects.org/ns/floodrisk/gauge_data#station_model_id#{}> <http://ensembleprojects.org/ns/floodrisk/gauge_data#station_model_name> "{}" .
                <http://ensembleprojects.org/ns/floodrisk/gauge_data#station_model_id#{}> <http://ensembleprojects.org/ns/floodrisk/gauge_data#station_model_company> "{}" .
                <http://ensembleprojects.org/ns/floodrisk/gauge_data#station_model_id#{}> <http://ensembleprojects.org/ns/floodrisk/gauge_data#station_model_date> "{}" .
              }} 
            }} 
            """.format(result['s']['value'],modelID,modelID,modelName,modelID,modelCompany,modelID,dateString)
    #sparql.setQuery(sparql_update)
    #results = sparql.query().convert()
    print(sparql_update)


### Example 8
Using the new triples generated in Example 7, we can now query each gauging station and find the details of the models used.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql_query = """
SELECT ?s ?name ?company ?date
FROM <http://ensembleprojects.org/gauge>
WHERE {
  ?s <http://ensembleprojects.org/ns/floodrisk/gauge_data#station_model_id> ?o .
  ?o <http://ensembleprojects.org/ns/floodrisk/gauge_data#station_model_name> ?name .
  ?o <http://ensembleprojects.org/ns/floodrisk/gauge_data#station_model_company> ?company .
  ?o <http://ensembleprojects.org/ns/floodrisk/gauge_data#station_model_date> ?date
}
"""
sparql = SPARQLWrapper(sparql_endpoint)
sparql.setReturnFormat(JSON)
sparql.setQuery(sparql_query)
results = sparql.query().convert()
for result in results['results']['bindings']:
    print(result['s']['value'], result['name']['value'], result['company']['value'], result['date']['value'])

### Example 9
Now that we have access to the model dates, we can easily query for those models that we suspect might be out of date.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql_query = """
PREFIX  xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX  gd:      <http://ensembleprojects.org/ns/floodrisk/gauge_data#>
SELECT ?station_name ?model_name ?date
FROM <http://ensembleprojects.org/gauge>
WHERE {
  ?station_name gd:station_model_id ?id .
  ?id gd:station_model_name ?model_name .
  ?id gd:station_model_date ?date .
  FILTER (xsd:date(?date) < "2007-01-01"^^xsd:date)
}
"""
sparql = SPARQLWrapper(sparql_endpoint)
sparql.setReturnFormat(JSON)
sparql.setQuery(sparql_query)
results = sparql.query().convert()
for result in results['results']['bindings']:
    print(result['station_name']['value'], result['model_name']['value'], result['date']['value'])

### Example 10



In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import matplotlib.pyplot as plt

def getQueryString(n):
    sparql_query = """
    prefix p:       <http://ensembleprojects.org/ns/floodrisk/property#>
    prefix pd:      <http://ensembleprojects.org/ns/floodrisk/property_data#>

    select *
    from <http://ensembleprojects.org/floodrisk/property>
    where {{
        p:{} pd:q5_gauge ?q5_gauge ;
            pd:q10_gauge ?q10_gauge ;
            pd:q20_gauge ?q20_gauge ;
            pd:q50_gauge ?q50_gauge ;
            pd:q75_gauge ?q75_gauge ;
            pd:q100_gauge ?q100_gauge ;
            pd:q200_gauge ?q200_gauge ;
            pd:q1000_gauge ?q1000_gauge ;
            pd:q5_existingLevelMean ?q5_existingLevelMean ;
            pd:q10_existingLevelMean ?q10_existingLevelMean ;
            pd:q20_existingLevelMean ?q20_existingLevelMean ;
            pd:q20_existingLevelMean ?q20_existingLevelMean ;
            pd:q50_existingLevelMean ?q50_existingLevelMean ;
            pd:q75_existingLevelMean ?q75_existingLevelMean ;
            pd:q100_existingLevelMean ?q100_existingLevelMean ;
            pd:q200_existingLevelMean ?q200_existingLevelMean ;
            pd:q1000_existingLevelMean ?q1000_existingLevelMean ;
            pd:property_thresh ?property_thresh .
    }}
    """.format(n)
    return sparql_query

def getMetaDataString(n):
    sparql_metadata = """
        prefix p:       <http://ensembleprojects.org/ns/floodrisk/property#>
        prefix pd:      <http://ensembleprojects.org/ns/floodrisk/property_data#>
        prefix gd:      <http://ensembleprojects.org/ns/floodrisk/gauge_data#>
        prefix damage: <http://ensembleprojects.org/ns/floodrisk/damage#>

        select ?model_name ?company ?date ?modelsource ?propertytype ?gauge
        from <http://ensembleprojects.org/floodrisk/property>
        from <http://ensembleprojects.org/floodrisk/gauge>
        from <http://ensembleprojects.org/gauge>
        where {{
            p:{} pd:model ?model ;
                 pd:modelSource ?modelsource ;
                 pd:jbapropertyType ?propertytype ;
                 pd:station_name ?gauge ;
                 pd:gauge_siteRef ?gauge_siteref .
            ?s gd:site_reference ?gauge_siteref ;
               gd:station_model_id ?id .
            ?id gd:station_model_name ?model_name ;
                gd:station_model_company ?company ;
                gd:station_model_date ?date 
            
        }}
    """.format(n)
    return sparql_metadata

# Assumes OpenLink Virtuoso sparql endpoint running at:
sparql_endpoint = "http://localhost:8890/sparql"

property = 24
sparql_query = getQueryString(property)
sparql = SPARQLWrapper(sparql_endpoint)
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
val_array = []
depth_array = []
for result in results["results"]["bindings"]:
    for k, v in result.items():
        if k == 'property_thresh':
            property_threshold = float(result[k]['value'])
        else:
            val_array.append(float(result[k]['value']))
gauge_array = val_array[:int(len(val_array)/2)]
flood_array = val_array[int(len(val_array)/2):]
for d in flood_array:
    f = d-property_threshold
    if f > 0:
        depth_array.append(d-property_threshold)
    else:
        depth_array.append(0.0)
plt.plot(depth_array,gauge_array,'--bo')
plt.xlabel("Property Flood Height")
plt.ylabel("Gauge Level")
plt.show()

sparql_query = getMetaDataString(property)
sparql.setQuery(sparql_query)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print("Model Study:\t", result['model_name']['value'])
    print("Organisation:\t", result['company']['value'])
    print("Date:\t\t", result['date']['value'])
    print("Model Source:\t", result['modelsource']['value'])
    print("Property Type:\t", result['propertytype']['value'])
    print("Gauge Station:\t", result['gauge']['value'])

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
import numpy as np

import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

def getQueryString(n):
    sparql_query = """
    prefix p:       <http://ensembleprojects.org/ns/floodrisk/property#>
    prefix pd:      <http://ensembleprojects.org/ns/floodrisk/property_data#>

    select *
    from <http://ensembleprojects.org/floodrisk/property>
    where {{
        p:{} pd:q5_gauge ?q5_gauge ;
            pd:q10_gauge ?q10_gauge ;
            pd:q20_gauge ?q20_gauge ;
            pd:q50_gauge ?q50_gauge ;
            pd:q75_gauge ?q75_gauge ;
            pd:q100_gauge ?q100_gauge ;
            pd:q200_gauge ?q200_gauge ;
            pd:q1000_gauge ?q1000_gauge ;
            pd:q5_existingLevelMean ?q5_existingLevelMean ;
            pd:q10_existingLevelMean ?q10_existingLevelMean ;
            pd:q20_existingLevelMean ?q20_existingLevelMean ;
            pd:q20_existingLevelMean ?q20_existingLevelMean ;
            pd:q50_existingLevelMean ?q50_existingLevelMean ;
            pd:q75_existingLevelMean ?q75_existingLevelMean ;
            pd:q100_existingLevelMean ?q100_existingLevelMean ;
            pd:q200_existingLevelMean ?q200_existingLevelMean ;
            pd:q1000_existingLevelMean ?q1000_existingLevelMean ;
            pd:property_thresh ?property_thresh .
    }}
    """.format(n)
    return sparql_query

def getMetaDataString(n):
    sparql_metadata = """
    prefix p:       <http://ensembleprojects.org/ns/floodrisk/property#>
    prefix pd:      <http://ensembleprojects.org/ns/floodrisk/property_data#>
    prefix gd:      <http://ensembleprojects.org/ns/floodrisk/gauge_data#>
    prefix damage:  <http://ensembleprojects.org/ns/floodrisk/damage#>

    select ?model_name ?company ?date ?modelsource ?propertytype ?gauge ?ead_mean
    from <http://ensembleprojects.org/floodrisk/property>
    from <http://ensembleprojects.org/floodrisk/gauge>
    from <http://ensembleprojects.org/gauge>
    from <http://ensembleprojects.org/floodrisk/damage#0>
    where {{
    p:{} pd:model ?model ;
        pd:modelSource ?modelsource ;
        pd:jbapropertyType ?propertytype ;
        pd:jbapropRef ?propertyref ;
        pd:station_name ?gauge ;
        pd:gauge_siteRef ?gauge_siteref .
    ?s gd:site_reference ?gauge_siteref ;
        gd:station_model_id ?id .
    ?id gd:station_model_name ?model_name ;
        gd:station_model_company ?company ;
        gd:station_model_date ?date .
    ?ds damage:jbapropRef ?propertyref ;
        damage:ead_mean ?ead_mean 
        }}
    """.format(n)
    return sparql_metadata

# Assumes OpenLink Virtuoso sparql endpoint running at:
sparql_endpoint = "http://localhost:8890/sparql"

property = 24

sparql_query = getQueryString(property)
sparql = SPARQLWrapper(sparql_endpoint)
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
val_array = []
depth_array = []
for result in results["results"]["bindings"]:
    for k, v in result.items():
        if k == 'property_thresh':
            property_threshold = float(result[k]['value'])
        else:
            val_array.append(float(result[k]['value']))
gauge_array = val_array[:int(len(val_array)/2)]
flood_array = val_array[int(len(val_array)/2):]
for d in flood_array:
    f = d-property_threshold
    if f > 0:
        depth_array.append(d-property_threshold)
    else:
        depth_array.append(0.0)
plt.plot(depth_array,gauge_array,'--bo', label="Piecewise Linear")
plt.xlabel("Property Flood Height")
plt.ylabel("Gauge Level")

depth_array_2d = np.reshape(depth_array, (-1, 1))
gauge_array_2d = np.reshape(gauge_array, (-1, 1))
regr = linear_model.LinearRegression()
regr.fit(depth_array_2d,gauge_array_2d)
predictions = regr.predict(depth_array_2d)
plt.plot(depth_array,predictions.flatten(),'--r', label="Linear Regression")
plt.legend(loc='upper left')
plt.show()

sparql_query = getMetaDataString(property)
sparql.setQuery(sparql_query)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print("Model Study:\t", result['model_name']['value'])
    print("Organisation:\t", result['company']['value'])
    print("Date:\t\t", result['date']['value'])
    print("Model Source:\t", result['modelsource']['value'])
    print("Property Type:\t", result['propertytype']['value'])
    print("Gauge Station:\t", result['gauge']['value'])
    print("\nEAD Mean:\t\t", result['ead_mean']['value'])
    
sparql_query = """
prefix damage:     <http://ensembleprojects.org/ns/floodrisk/damage#>
prefix damageData: <http://ensembleprojects.org/ns/floodrisk/damage_data#>

select *
from <http://ensembleprojects.org/floodrisk/damageestimates> 
where {
 ?s damageData:dataGraph <http://ensembleprojects.org/floodrisk/damage#0> ;
    damageData:method ?method ;
    damageData:interpolationScheme ?interpolationScheme ;
    damageData:numRuns ?numRuns ;
    damageData:uncertainty ?uncertainty ;
    damageData:numSimulations ?numSimulations ;
    damageData:software ?software;
    damageData:date ?date .
 
}
"""
sparql.setQuery(sparql_query)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print("EAD Method:\t\t", result['method']['value'])
    print("EAD Interpolation:\t", result['interpolationScheme']['value'])
    print("EAD Number of Runs:\t", result['numRuns']['value'])
    print("EAD Uncertainty:\t", result['uncertainty']['value'])
    print("EAD No. of Simulations:\t", result['numSimulations']['value'])
    print("EAD Software:\t\t", result['software']['value'])
    print("EAD Date:\t\t", result['date']['value'])
    

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

sparql_query = """
prefix g:       <http://ensembleprojects.org/ns/floodrisk/gauge#>
prefix gd:      <http://ensembleprojects.org/ns/floodrisk/gauge_data#>
prefix p:       <http://ensembleprojects.org/ns/floodrisk/property#>
prefix pd:      <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix fr:      <http://ensembleprojects.org/floodrisk/>

select ?name ?flow ?val
from <http://ensembleprojects.org/floodrisk/gauge>
from <http://ensembleprojects.org/floodrisk/property>
where {
        g:0 gd:station_name ?name .
        g:0 ?flow ?val .
        ?flow a fr:q .
}
"""
# Assumes OpenLink Virtuoso sparql endpoint running at:
sparql_endpoint = "http://localhost:8890/sparql"

sparql = SPARQLWrapper(sparql_endpoint)
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

df = pd.DataFrame(results['results']['bindings'])

for index, row in df.iterrows():
    print(row['flow'])



#for result in results["results"]["bindings"]:
#   print (result["flow"]["value"], result["name"]["value"], result["val"]["value"])

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql_query = """
select * 
from <http://ensembleprojects.org/floodrisk/damageestimates>
where {
  ?s ?p ?o
}
"""
# Assumes OpenLink Virtuoso sparql endpoint running at:
sparql_endpoint = "http://localhost:8890/sparql"

sparql = SPARQLWrapper(sparql_endpoint)
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print (result["s"]["value"], result["p"]["value"], result["o"]["value"])

In [ ]:
PREFIX pd: <http://ensembleprojects.org/ns/floodrisk/property_data#>
      SELECT distinct ?gauge_siteRef ?station_name
      FROM <http://ensembleprojects.org/floodrisk/property>
      WHERE {
        ?property pd:posttown "NEWARK" .
        ?property pd:gauge_siteRef ?gauge_siteRef .
        ?property pd:station_name ?station_name
}